In [ ]:
import numpy as np
import matplotlib.pyplot as plt

""" function used to loosely model our prior belief about the aerosol structure, spikes represented by the sin component,
     diurnal and montly cycles represented by the cos components repectively """

def target_func_2d(x):
    y = np.sin( 4 * x) ** 3 + np.cos( 6 * x) + np.cos( (1/6) * x)
    epsilon = np.random.normal(0, 1e-4)

    return y + epsilon

# generate points and plot

x = np.random.uniform(0, 30, 400).reshape(1,-1)
y = target_func_2d(x).reshape(1,-1)

print(x, y)
# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Noisy Function', color='red', s=10, alpha=0.6)
plt.xlabel('x')
plt.ylabel('y')
plt.title('True Function vs Noisy Function with Aerosol Structure')
plt.legend()
plt.grid(True)
plt.show()

In [22]:
import torch
import gpytorch
from sklearn.metrics import mean_squared_error

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.PeriodicKernel() + gpytorch.kernels.MaternKernel(nu=2.5)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def optimize_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)

    training_iter = 200
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    loss_history = []

    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y).mean()
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())

    return model, likelihood, loss_history

def evaluate_model(model, likelihood, test_x, test_y):
    model.eval()
    likelihood.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(test_x))
        
        # Mean Squared Error
        mse = mean_squared_error(test_y.numpy(), observed_pred.mean.numpy())
        
        # Mean Interval Score
        lower, upper = observed_pred.confidence_region()
        mis = ((upper - lower) + (2 / 0.05) * ((lower - test_y).clamp(min=0) + (test_y - upper).clamp(min=0))).mean().item()
        
    return mse, mis, observed_pred, lower, upper

def plot_results(test_x, test_y, observed_pred, lower, upper):
    plt.figure(figsize=(10, 6))

    # Plot the actual function
    x = np.linspace(0, 6 * np.pi, 1000)
    y = np.sin(4 * x) ** 3 + np.cos(6 * x) + np.cos((1 / 6) * x)
    plt.plot(x, y, label='Actual Function', color='blue')

    # Plot the predictions, test points, and confidence interval
    plt.plot(test_x.numpy(), observed_pred.mean.numpy(), 'k*', label='Predictions')
    plt.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5, label='Confidence Interval')
    plt.scatter(test_x.numpy(), test_y.numpy(), label='Test Points', color='red')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Actual Function vs Predictions with Confidence Interval')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
x = x.torch